# Grocery forecasting data merger

This notebook serves as a data merger where all the features from the tables other than `train.csv` and `test.csv` get merged into them. The result being a `train.csv` and `test.csv` with more feature columns than they intially had.

<p style="color:red;">/!\ Be aware that this is not the most efficient way to join the columns as it will consume a lot of memory and the resulting csv will be very large (more on this below). You may want to take that code and adapt it so that `train.csv` get read by chunks</p>

## 1. Open & view the data

In [ ]:
import sys
import os
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import datetime

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
path = "../input/"
files = ["train.csv", "test.csv", "transactions.csv" , "stores.csv", "oil.csv", "items.csv", "holidays_events.csv"]
datasets_path = [path + f for f in files]
print('# File sizes')
for f in datasets_path:
    print(f.ljust(30) + str(round(os.path.getsize(f) / 1000000, 2)) + 'MB')

Notice we use a `chunksize` to not load all the dataset on Kaggle virtual machine. You may want to set it to `None` on your machine.

We define the type for each tables because by default pandas loads the csv with float64/int64 columns which will doubles the size of our resulting csv.

This notebook assumes you have enough memory to handle all the operations/creation of the different datasets we will have. A better alternative could be to load a chunk of the csv file then iteratively store the chunk in the resulting csv. It's a hard thing to do in a notebook but you can take most of the code here to achieve this goal.

**The training set has 125 497 040 entries and the the testing set has 3 370 464 entries in total.**

<p style="color:red;">To run this notebook on all the data you will need to have at least 7gb of free RAM as well as 1.2gb of free disk space</p>

In [ ]:
%%time

# This will take all the data from the test set but not the train set
# It's ~15x less data from the train set 
chunksize = 8_366_470

data_df = {}
for file, path in tqdm(zip(files, datasets_path), total=len(files)):
    name = file.split(".")[0]
    if name == 'train' or name == 'test':
        data_df[name] = pd.read_csv(path, dtype={"date": np.str,
                                                 "id": np.int32,
                                                 "item_nbr": np.int32,
                                                 "onpromotion": np.object,
                                                 "store_nbr": np.int8,
                                                 "unit_sales": np.float32},
                                    parse_dates=["date"], chunksize=chunksize, 
                                    low_memory=False)
        if chunksize:
            data_df[name] = data_df[name].get_chunk()
        data_df[name]["onpromotion"].fillna(False, inplace=True)
        data_df[name]["onpromotion"].map({"True": True, "False": False})
        data_df[name]["onpromotion"].astype(bool)
    elif name == 'transactions':
        data_df[name] = pd.read_csv(path, dtype={"date": np.str,
                                                 "store_nbr": np.int32,
                                                 "transactions": np.int32},
                                   parse_dates=["date"])
    elif name == 'stores':
        data_df[name] = pd.read_csv(path, dtype={"store_nbr": np.int8,
                                                 "city": np.str,
                                                 "state": np.str,
                                                 "type": np.str,
                                                 "cluster": np.int32})
    elif name == 'oil':
        data_df[name] = pd.read_csv(path, dtype={"date": np.str,
                                                 "dcoilwtico": np.float32},
                                   parse_dates=["date"])
    elif name == 'oil':
        data_df[name] = pd.read_csv(path, dtype={"date": np.str,
                                                 "dcoilwtico": np.float32},
                                   parse_dates=["date"])
    elif name == 'items':
        data_df[name] = pd.read_csv(path, dtype={"item_nbr": np.int32,
                                                 "family": np.str,
                                                 "class": np.str,
                                                 "perishable": np.bool})
    elif name == 'holidays_events':
        data_df[name] = pd.read_csv(path, dtype={"date": np.str,
                                                 "type": np.str,
                                                 "locale": np.str,
                                                 "locale_name": np.str,
                                                 "description": np.str,
                                                 "transferred": np.bool},
                                   parse_dates=["date"])
    else:
        data_df[name] = pd.read_csv(path)

In [ ]:
for k, df in data_df.items():
    print(k)
    display(df.head())

In [ ]:
train_df = data_df["train"]
test_df = data_df["test"]
oil_df = data_df["oil"]
oil_df["date"] = pd.to_datetime(oil_df["date"])
items_df = data_df["items"]
stores_df = data_df["stores"]
transactions_df = data_df["transactions"]
transactions_df["date"] = pd.to_datetime(transactions_df["date"])
holidays_df = data_df["holidays_events"]
holidays_df["date"] = pd.to_datetime(holidays_df["date"])

print("Train set date range: {} to {}".format(train_df["date"].min(), train_df["date"].max()))
print("Test set date range: {} to {}".format(test_df["date"].min(), test_df["date"].max()))

## 2. Merge train/test

Now lets merge train and test sets, so all operations can be operated onto 1 Dataframe, we can split them back later as we know the train set range from **2013-01-01 to 2017-08-15** and the test set from **2017-08-16 to 2017-08-31**.

In [ ]:
train_df['onpromotion'].fillna(False, inplace=True)
test_df['onpromotion'].fillna(False, inplace=True)

train_date_range = [train_df['date'].min(), train_df['date'].max()]
test_date_range = [test_df['date'].min(), test_df['date'].max()]

merged_df = train_df.append(test_df)
origin_merge_len = len(merged_df)
origin_train_len = len(train_df)
origin_test_len = len(test_df)

assert len(merged_df) == len(train_df) + len(test_df)
print("Merged df size: {}".format(len(merged_df)))
del train_df
del test_df
gc.collect()

display(merged_df.tail())
display(merged_df.dtypes)
print(f"Train set range:{train_date_range}\nTest set range:{test_date_range}")

In [ ]:
display(merged_df.isnull().sum().sort_index() / len(merged_df))

We can see there are no missing values in the train/test sets as the missing values from `unit_sales` are the ones from the test set.

Now lets merge all the datasets

## 3. Merge each table into our train/test dataframe

### 3.1 Transactions

From Kaggle:
> The count of sales transactions for each date, store_nbr combination. Only included for the training data timeframe.

<p style="color:red;">/!\ There is no dates with transactions for the test set</p>

In [ ]:
merged_df = merged_df.merge(transactions_df, on=["date", "store_nbr"], how='left')
merged_df.head()
print(transactions_df['date'].max())

We merge on [left](http://www.datacarpentry.org/python-ecology-lesson/04-merging-data/) because we want to preserve all the rows of the test set which don't have any couple `["date", "store_nbr"]` for the test set

In [ ]:
print(len(merged_df))
display(merged_df.head())
display(merged_df.tail())

### 3.2 Stores
From Kaggle:
> Store metadata, including `city`, `state`, `type`, and `cluster`. `cluster` is a grouping of similar stores.

In [ ]:
stores_df.columns = ['store_' + col if col != "store_nbr" else "store_nbr" for col in stores_df.columns]
stores_df.head(3)

In [ ]:
merged_df = merged_df.merge(stores_df, on=["store_nbr"])

In [ ]:
print(len(merged_df))
display(merged_df.head())
display(merged_df.tail())

### 3.3 Oil

The oild index exists for both the train and test sets but the oil index is not available for every date of the merged dataset

In [ ]:
oil_df.tail(3)

In [ ]:
merged_df = merged_df.merge(oil_df, on=["date"], how='left')

In [ ]:
assert len(merged_df) == origin_merge_len
display(merged_df.head())
display(merged_df.tail())

### 3.4 Items

From Kaggle:
> Item metadata, including family, class, and perishable.
>
> NOTE: Items marked as perishable have a score weight of 1.25; otherwise, the weight is 1.0.

In [ ]:
items_df.columns = ['item_' + col if col != "item_nbr" else "item_nbr" for col in items_df.columns]
items_df.head(3)

In [ ]:
merged_df = merged_df.merge(items_df, on=["item_nbr"])

In [ ]:
assert len(merged_df) == origin_merge_len
display(merged_df.head())
display(merged_df.tail())

### 3.5 Holidays events
<p style="color:red;"> There are duplicate dates in the `date` column, we need to fix this before merging `holidays_df` into `merged_df`.</p>

In [ ]:
print(f"# of entries: {len(holidays_df)}")
print(holidays_df.nunique())
holidays_df[holidays_df['date'].duplicated(keep=False)].head()

We can see that these duplications are due to the fact that some holidays of the same day can be different depending on the `locale_name`. <p style="color:red;"> So here we need to be careful to not only merge the datasets on the `date` but also on the `locale_name` where the `store` of a given entry is located.</p>

To make things easier here we won't try to map `store_city` or `store_state` from `merged_df` to the `locale_name` of `holidays_df`. We'll just get rid of every rows which `locale_name` is not `Ecuador` and check if we still have duplicates.

In [ ]:
holidays_df = holidays_df[holidays_df["locale_name"] == "Ecuador"]
print(f"# of entries: {len(holidays_df)}")
print(holidays_df.nunique())
holidays_df[holidays_df['date'].duplicated(keep=False)].head()

Not yet! We still have duplicates, so lets just remove the duplicates now

In [ ]:
holidays_df = holidays_df[np.invert(holidays_df['date'].duplicated())]
print(f"# of entries: {len(holidays_df)}")
print(holidays_df.nunique())
holidays_df.head()

Great we're done with the duplicates, now we can now continue drop `locale` and `locale_name` as they all have the same value.

In [ ]:
holidays_df = holidays_df.drop(["locale", "locale_name"], axis=1)

Now to the merge.

In [ ]:
holidays_df.columns = ['holiday_' + col if col != "date" else "date" for col in holidays_df.columns]
holidays_df.head(3)

Here we'll do something a bit different. According to Kaggle:
> A holiday that is `transferred` officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where `type` is `Transfer`. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.

So basically we are only interested in the day when the holiday event actually happened. Before merging this table to our `merged_df` we can get rid of the rows where `transferred` is `True` and then get rid of the column itself.

In [ ]:
holidays_df = holidays_df[holidays_df["holiday_transferred"] != True]
holidays_df = holidays_df.drop(["holiday_transferred"], axis=1)
holidays_df.head()

In [ ]:
merged_df = merged_df.merge(holidays_df, on="date", how='left')

In [ ]:
assert len(merged_df) == origin_merge_len
display(merged_df.head())
display(merged_df.tail())

## Final datasets output & checkup

Lets take a look at what our final dataset looks like

In [ ]:
print(len(merged_df))
display(merged_df.dtypes)
merged_df.head()

Now lets define our `id` as index

In [ ]:
merged_df.set_index("id", inplace=True)

Do we still have the same number of entry as before we started the merging operations?

In [ ]:
assert len(merged_df) == origin_merge_len

Count the missing values for each set on the whole

In [ ]:
display(merged_df.isnull().sum().sort_index() / len(merged_df))

Looks legit!

Now we can get rid of columns we know are completely useless such as `item_nbr`, `store_nbr` which are just identifiers and does not hold relevant informations.

In [ ]:
merged_df = merged_df.drop(["item_nbr", "store_nbr"], axis=1)

Now lets split back to test/train splits

In [ ]:
train_df = merged_df[(merged_df['date'] >= train_date_range[0]) & (merged_df['date'] <= train_date_range[1])]
train_df = train_df.sort_index()
test_df = merged_df[(merged_df['date'] >= test_date_range[0]) & (merged_df['date'] <= test_date_range[1])]
test_df = test_df.sort_index()
print(f"train_df range: {train_df['date'].min()} to {train_df['date'].max()}")
print(f"test_df range: {test_df['date'].min()} to {test_df['date'].max()}")

In [ ]:
assert origin_train_len == len(train_df)
assert origin_test_len == len(test_df)
display(train_df.head(3))
display(test_df.head(3))

And finally save them as csv:

In [ ]:
# train_df.to_csv("train_joined.csv")
# test_df.to_csv("test_joined.csv")
print("Operation finished!")

<p style="color:red;">Don't forget to shutdown your kernel now to clear up your memory</p>

Now we can do some feature engineering and transform our data to a format usable by our machine learning models. 
We could also exploit this information from kaggle to scrape useful informations from the net to add to our dataset:
> Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.

So maybe every 15/16/17th the sales get higher a bit?
But we'll leave this here for now :)

Happy hacking!